In [39]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
from scipy.stats import pearsonr
import plotnine as p9
import plotly.express as px
import os
import geopy.distance
import wget

In [40]:
# let's load 1 fcm dataset for now- underway g3
ug3 = pd.read_feather('/Users/Kathy/Desktop/UW/colocalization/gradients_individual/ug3_FCM_distribution.feather')
# load ecotype dataand metadata
eco = pd.read_csv('/Users/Kathy/Desktop/UW/colocalization/gradients_pro_syn_relative_abundance.csv', 
                 usecols = ['SampleID','Genus','Clade','RelAbundance'])
eco_meta = pd.read_csv('/Users/Kathy/Desktop/UW/colocalization/sample_metadata.csv', 
                      usecols=['SampleID','Cruise','Station','Datetime','Latitude','Longitude','Depth',
                              'Replicate'])
# align data columns
eco_meta['Datetime'] = eco_meta['Datetime'].apply(lambda x: parse(x))
eco_meta['Cruise'] = eco_meta['Cruise'].str.upper()
# filter out data with no location data (inc experiments)
sub_eco_meta = eco_meta[(eco_meta['Latitude'].notnull()) & (eco_meta['Depth'].notnull())]
# merge with metadata
merged_eco = sub_eco_meta.merge(eco, how='left', on='SampleID')

In [41]:
# relative abundance is to the total number of cells in the sample (including pro/syn)
# calculate the total relative abundance to genus 
grouped = merged_eco.groupby(['SampleID','Genus'])
adj_eco = []
for s,new_df in grouped:
    sum_all = np.sum(new_df['RelAbundance'])
    new_df['RelAbundance_adj'] = new_df['RelAbundance']/sum_all
    # save into new dataframe
    adj_eco.append(new_df)
RA_eco = pd.concat(adj_eco).reset_index().drop(columns=['index'])

In [45]:
grouped_fcm = ug3.groupby(['filename','date','lat','lon','depth']).mean().reset_index()
px.scatter(grouped_fcm, x='lat',y='diam_mid', color='norm.red')

In [38]:
g3_eco = RA_eco[RA_eco['Cruise']=='KM1906']
g3_eco[g3_eco['pop']]

,SampleID,Cruise,Station,Datetime,Latitude,Longitude,Depth,Replicate,Genus,Clade,RelAbundance,RelAbundance_adj
1550,G3.DEPTH.NS.S4C6.15m.A,KM1906,4.0,2019-04-16 08:11:00,41.67,-158.05,15.0,A,Prochlorococcus,HLI,0.002480,0.020068
1551,G3.DEPTH.NS.S4C6.15m.A,KM1906,4.0,2019-04-16 08:11:00,41.67,-158.05,15.0,A,Prochlorococcus,HLII,0.084844,0.686617
1552,G3.DEPTH.NS.S4C6.15m.A,KM1906,4.0,2019-04-16 08:11:00,41.67,-158.05,15.0,A,Prochlorococcus,HLIII,0.000000,0.000000
1553,G3.DEPTH.NS.S4C6.15m.A,KM1906,4.0,2019-04-16 08:11:00,41.67,-158.05,15.0,A,Prochlorococcus,HLVI,0.000046,0.000376
1554,G3.DEPTH.NS.S4C6.15m.A,KM1906,4.0,2019-04-16 08:11:00,41.67,-158.05,15.0,A,Prochlorococcus,LLI,0.025648,0.207559
...,...,...,...,...,...,...,...,...,...,...,...,...
3820,G3.UW.NS.UW40_2.7m.C,KM1906,NaN,2019-04-25 06:01:00,40.09,-158.00,7.0,C,Synechococcus,5.1B-VIII,0.000019,0.000023
3821,G3.UW.NS.UW40_2.7m.C,KM1906,NaN,2019-04-25 06:01:00,40.09,-158.00,7.0,C,Synechococcus,5.2,0.000276,0.000332
3822,G3.UW.NS.UW40_2.7m.C,KM1906,NaN,2019-04-25 06:01:00,40.09,-158.00,7.0,C,Synechococcus,5.3-I / X,0.000000,0.000000
3823,G3.UW.NS.UW40_2.7m.C,KM1906,NaN,2019-04-25 06:01:00,40.09,-158.00,7.0,C,Synechococcus,CDR2,0.006409,0.007710


In [ ]:
px.scatter(RA_eco, x='lat',y='depth',)